In [1]:
# working folder: Sandbox

In [ ]:
📦 PhotoMaps_Pipeline.ipynb
 ├── 🔍 1. Исходные данные
 │    ├── Чтение данных из папки images/
 │    ├── Удаление дубликатов
 │    ├── Конвертация HEIC → JPG
 │    └── Анализ данных (JPG, JPEG, HEIC, JSON)
 │
 ├── 📝 2. Генерация CSV файлов
 │    ├── photos.csv — Основной CSV
 │    ├── missing_coords.csv — Файлы без координат
 │    └── import_log.txt — Лог импорта
 │
 ├── 🌐 3. Обновление ссылок Google Drive
 │    ├── Авторизация Google API
 │    ├── Сбор ID файлов в google_drive_links.csv
 │    └── Создание прямых ссылок
 │
 ├── 🌍 4. Создание GeoJSON
 │    ├── Преобразование данных из photos.csv → photos.geojson
 │    ├── Добавление country_code
 │    └── Добавление источника source_type
 │
 └── 📍 5. Обновление GeoJSON
      ├── Добавление `photo_link`, `thumb_url`, `description`
      └── Создание photos_updated.geojson

In [ ]:
# 1. Study folder images/ and get all the stats

In [21]:
import os
import shutil
from collections import Counter
from PIL import Image

# === Настройки ===
SANDBOX_DIR = 'sandbox/'
SANDBOX_IMAGES_FOLDER = os.path.join(SANDBOX_DIR, 'images_full/')
CONVERTED_FOLDER = os.path.join(SANDBOX_DIR, 'images_converted/')
CSV_FOLDER = os.path.join(SANDBOX_DIR, 'csv/')
JSON_FOLDER = os.path.join(SANDBOX_DIR, 'json/')
THUMBNAILS_FOLDER = os.path.join(SANDBOX_DIR, 'thumbnails/')
CSS_FOLDER = os.path.join(SANDBOX_DIR, 'css/')
VALID_EXTENSIONS = ['.jpg', '.jpeg', '.heic', '.json']

# === Счетчики ===
file_counter = Counter()
duplicate_counter = 0
heic_files = []
invalid_jsons = []

# === Создание папок ===
os.makedirs(SANDBOX_IMAGES_FOLDER, exist_ok=True)
os.makedirs(CONVERTED_FOLDER, exist_ok=True)
os.makedirs(CSV_FOLDER, exist_ok=True)
os.makedirs(JSON_FOLDER, exist_ok=True)
os.makedirs(THUMBNAILS_FOLDER, exist_ok=True)
os.makedirs(CSS_FOLDER, exist_ok=True)

# === 1️⃣ Сканирование песочницы ===
def scan_sandbox():
    global duplicate_counter
    print("🔍 Сканирование песочницы sandbox/images_full/...")
    for filename in os.listdir(SANDBOX_IMAGES_FOLDER):
        ext = os.path.splitext(filename.lower())[1]
        if ext in VALID_EXTENSIONS:
            file_counter[ext] += 1
            
            # Сбор HEIC для конвертации
            if ext == '.heic':
                heic_files.append(filename)
                
            # Поиск дубликатов
            if '(1)' in filename or '(2)' in filename or 'copy' in filename.lower():
                duplicate_counter += 1
            
            # Поиск некорректных JSON
            if ext == '.json' and ('(1)' in filename or '(2)' in filename):
                invalid_jsons.append(filename)

# === 2️⃣ Удаление дубликатов ===
def remove_duplicates(folder):
    print("🗑️ Удаление дубликатов...")
    to_remove = [f for f in os.listdir(folder) if '(1)' in f or '(2)' in f or 'copy' in f.lower()]
    for filename in to_remove:
        os.remove(os.path.join(folder, filename))
        print(f"❌ Удален дубликат: {filename}")

# === 3️⃣ Конвертация HEIC → JPG ===
def convert_heic_to_jpg():
    print("🔄 Конвертация HEIC → JPG...")
    for heic_file in heic_files:
        base_filename = os.path.splitext(heic_file)[0]
        jpg_filename = f"{base_filename}.jpg"
        try:
            with Image.open(os.path.join(SANDBOX_IMAGES_FOLDER, heic_file)) as img:
                img.convert("RGB").save(os.path.join(CONVERTED_FOLDER, jpg_filename), "JPEG")
            print(f"✅ Конвертировано: {heic_file} → {jpg_filename}")
        except Exception as e:
            print(f"⚠️ Ошибка при конвертации {heic_file}: {e}")

# === 4️⃣ Анализ папки ===
def analyze_sandbox():
    print("\n=== 📊 Результаты анализа ===")
    for ext, count in file_counter.items():
        print(f"{ext.upper()}: {count}")
    print(f"🗑️ Найдено дубликатов: {duplicate_counter}")
    print(f"⚠️ Неправильные JSON: {len(invalid_jsons)}")
    print(f"📸 HEIC для конвертации: {len(heic_files)}")
    print(f"✅ Успешно конвертировано: {len(os.listdir(CONVERTED_FOLDER))}")
    print("\n🔍 Анализ завершен!")

# === Запуск пайплайна ===
scan_sandbox()
remove_duplicates(SANDBOX_IMAGES_FOLDER)
convert_heic_to_jpg()
analyze_sandbox()

🔍 Сканирование песочницы sandbox/images_full/...
🗑️ Удаление дубликатов...
❌ Удален дубликат: IMG_20240518_142157622.jpg.supplemental-metada copy.json
❌ Удален дубликат: IMG_1095.JPG.supplemental-metadata(1).json
❌ Удален дубликат: IMG_20240519_111852521_HDR copy.jpg
❌ Удален дубликат: IMG_1095(1).JPG
❌ Удален дубликат: IMG_1071.JPG.supplemental-metadata(1).json
❌ Удален дубликат: IMG_1071(1).JPG
❌ Удален дубликат: IMG_20240518_142157622 copy.jpg
❌ Удален дубликат: IMG_20240519_111852521_HDR.jpg.supplemental-me copy.json
🔄 Конвертация HEIC → JPG...

=== 📊 Результаты анализа ===
.JPG: 18
.JSON: 17
🗑️ Найдено дубликатов: 8
⚠️ Неправильные JSON: 2
📸 HEIC для конвертации: 0
✅ Успешно конвертировано: 0

🔍 Анализ завершен!
